### 1. **Objetivos**

**Objetivo do Modelo**:
- O objetivo deste modelo é prever dois fatores críticos para a produção:
  
  1. **Total de defeitos** esperados para uma nova ordem de produção, com base em medidas físicas (peso, comprimento e diâmetro) e duração da nova ordem.
  
  2. **Quantidades de defeitos por dia**, prever em qual dia da produção será mais provável ocorrer o maior número de defeitos.

### 2. **Solução**

**Solução Proposta**:
- Para prever o **total de defeitos**, vou implementar um modelo de **regressão**, que será treinado com dados históricos sobre medidas físicas (peso, comprimento e diâmetro) e o total de defeitos observados em ordens anteriores.
- Para prever os **dias críticos de defeitos**, farei uma análise de sazonalidade, que considera o **padrão** de defeitos ao longo dos dias de produção de cada ordem. Isso nos ajudará a identificar quais dias (por exemplo, dia 2) têm maior chance de apresentar defeitos.
- Além disso, a **duração da ordem** (quantidade de dias de produção) será um parâmetro importante para ajustar as previsões. Ordens mais longas tendem a ter mais defeitos, e a duração influencia o comportamento dos defeitos ao longo dos dias.

**Algoritmos Utilizados**:
- **Regressão (Gradient Boosting)**: Para prever o **total de defeitos** com base nas características do pedido e a duração da ordem.
- **Análise de série temporal**: Para identificar **quais dias** terão maior probabilidade de defeitos ao longo da produção.

### 3. **Como o Modelo Vai Funcionar (Inputs e Outputs)**

**Inputs (Entradas do Modelo)**:
- **Medidas da Ordem**:
  - Peso médio (g)
  - Comprimento médio (mm)
  - Diâmetro
- **Duração da Ordem**:
  - Número de dias que a produção vai durar (fornecido pelo time de vendas ou orçamento).
- **Dados Históricos**: O modelo é treinado com ordens passadas, suas medidas e os defeitos observados.

**Outputs (Saídas do Modelo)**:
- **Previsão do Total de Defeitos**: Para uma nova ordem, o modelo preverá o total de defeitos, com base nas novas medidas fornecidas.
- **Previsão dos Dias Críticos de Defeitos**: O modelo também preverá os dias críticos de defeitos, ou seja, os dias específicos em que a ordem pode apresentar mais defeitos, com base no histórico de ordens anteriores e sua sazonalidade.

### 4. **Conclusão**

- **Benefícios**:
  
  - **Previsões mais precisas** de defeitos, com base em dados reais e a duração de cada ordem.
  
  - **Ação preventiva**: Saber antecipadamente quais dias têm mais chance de defeitos pode otimizar os processos de produção.
  
  - **Apoio à decisão**: Ajudar os times de produção e vendas a planejar melhor as ordens e a controlar a qualidade da produção.


### 5. **Próximos Passos**

Para aprimorar e expandir a solução proposta, as seguintes ações serão realizadas:  

1. **Treinamento do Modelo com Mais Dados**:  
   - Coletar e incluir mais dados históricos que apresentem uma relação clara entre as medidas da ordem, duração da produção e tipos de defeitos.  
   - Incorporar dados com maior granularidade, como características adicionais da produção (ex.: condições ambientais, materiais utilizados ou ajustes específicos na linha de produção).  

2. **Previsão Detalhada de Defeitos**:  
   - Modificar o modelo para prever não apenas o total e a distribuição dos defeitos, mas também **quais tipos específicos de defeitos podem ocorrer**.  
   - Fornecer insights sobre a **criticidade** de cada tipo de defeito, priorizando ações corretivas para os mais graves.  

3. **Aumento da Granularidade das Previsões**:  
   - Ajustar o modelo para fornecer previsões diárias mais detalhadas, como:  
     - **Quantidades específicas de cada defeito por dia**.  
     - **Probabilidade de ocorrência** de defeitos em determinados períodos (manhã, tarde ou noite).  

4. **Iteração e Validação Contínua**:  
   - Após incorporar novos dados, realizar ajustes e validações contínuas no modelo para garantir maior precisão e confiabilidade.  
   - Aplicar técnicas como validação cruzada e ajuste de hiperparâmetros para otimizar o desempenho do modelo.


## Implementação do Modelo

### Importando bibliotecas

In [71]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go

### Importando dataset

In [72]:
dados = pd.read_csv('../dados/medidas_geral_e_defeitos.csv')
df = pd.DataFrame(dados)

### Definindo multiíndice

In [73]:
df = df.set_index(['Ordem de prod', 'Ref. do Artigo', 'Data de prod'])

In [74]:
df

Temperatura (F°)  Peso médio (g)  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03             2126.97           169.3   
                             2024-01-04             2118.92           169.7   
                             2024-01-05             2125.00           173.6   
                             2024-01-06             2125.40           170.2   
                             2024-01-07             2126.76           171.7   
...                                                     ...             ...   
194121        C0516.0000R    2024-06-26             2154.44           169.0   
                             2024-06-27             2152.32           168.9   
                             2024-06-28             2145.60           169.3   
                             2024-06-29             2145.60           168.8   
                             2024-06-30             2148.00           169.2   

                                           Comprimento médio (mm)  Diâmetro  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03                     110.4      32.5   
                             2024-01-04                      85.6      18.1   
                             2024-01-05                     140.5      32.9   
                             2024-01-06                     144.7      32.7   
                             2024-01-07                     141.5      30.2   
...                                                           ...       ...   
194121        C0516.0000R    2024-06-26                     124.3      32.6   
                             2024-06-27                     120.2      33.6   
                             2024-06-28                     124.1      34.2   
                             2024-06-29                     120.0      33.5   
                             2024-06-30                     122.9      34.6   

                                           Gotas cortadas (toneladas)  \
Ordem de prod Ref. do Artigo Data de prod                               
192970        C0516.0000R    2024-01-03                        20.948   
                             2024-01-04                        21.022   
                             2024-01-05                        21.449   
                             2024-01-06                        21.526   
                             2024-01-07                        21.439   
...                                                               ...   
194121        C0516.0000R    2024-06-26                        21.412   
                             2024-06-27                        21.447   
                             2024-06-28                        21.491   
                             2024-06-29                        21.419   
                             2024-06-30                        21.548   

                                           Total de Defeitos no Dia  \
Ordem de prod Ref. do Artigo Data de prod                             
192970        C0516.0000R    2024-01-03                         222   
                             2024-01-04                         249   
                             2024-01-05                         268   
                             2024-01-06                         242   
                             2024-01-07                         232   
...                                                             ...   
194121        C0516.0000R    2024-06-26                         258   
                             2024-06-27                         218   
                             2024-06-28                         218   
                             2024-06-29                         213   
                             2024-06-30                         256   

                                          Defeito Mais Frequente do Dia  \
Ordem de prod Ref. do Artigo Data de p

### Verificando se o dataframe está limpo e padronizado

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 200 entries, (192970, 'C0516.0000R', '2024-01-03') to (194121, 'C0516.0000R', '2024-06-30')
Columns: 134 entries, Temperatura (F°) to Crú
dtypes: float64(5), int64(128), object(1)
memory usage: 217.8+ KB


In [76]:
df.isnull().sum()

Temperatura (F°)              0
Peso médio (g)                0
Comprimento médio (mm)        0
Diâmetro                      0
Gotas cortadas (toneladas)    0
                             ..
Ferrugem no corpo             0
Fagulha                       0
Alça deformada                0
Instabilidade                 0
Crú                           0
Length: 134, dtype: int64

### Criando Dataframe sem índice

In [77]:
df_reset = df.reset_index()

In [156]:
df_reset

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Gotas cortadas (toneladas),Total de Defeitos no Dia,Defeito Mais Frequente do Dia,...,Bolha aberta interna,Cascão cortante,Espessura abaixo do mínimo,Fundo deformado,Bolha na alça,Ferrugem no corpo,Fagulha,Alça deformada,Instabilidade,Crú
0,192970,C0516.0000R,2024-01-03,2126.97,169.3,110.4,32.5,20.948,222,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0
1,192970,C0516.0000R,2024-01-04,2118.92,169.7,85.6,18.1,21.022,249,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0
2,192970,C0516.0000R,2024-01-05,2125.00,173.6,140.5,32.9,21.449,268,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0
3,192970,C0516.0000R,2024-01-06,2125.40,170.2,144.7,32.7,21.526,242,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0
4,192970,C0516.0000R,2024-01-07,2126.76,171.7,141.5,30.2,21.439,232,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,194121,C0516.0000R,2024-06-26,2154.44,169.0,124.3,32.6,21.412,258,Sujo de óleo,...,1,0,0,0,0,0,0,0,0,0
196,194121,C0516.0000R,2024-06-27,2152.32,168.9,120.2,33.6,21.447,218,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0
197,194121,C0516.0000R,2024-06-28,2145.60,169.3,124.1,34.2,21.491,218,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0
198,194121,C0516.0000R,2024-06-29,2145.60,168.8,120.0,33.5,21.419,213,Sujo de óleo,...,0,0,0,0,0,0,1,0,0,0


### Variância dos defeitos por dia

In [ ]:
fig = px.box(
    df_reset,
    y='Total de Defeitos no Dia',
    title='Distribuição de Total de Defeitos no Dia',
    points='all',
    color_discrete_sequence=['#636EFA']
)

# Ajustar layout
fig.update_layout(
    yaxis_title="Total de Defeitos no Dia",
    xaxis_title="",
    title_font=dict(size=18),
    font=dict(size=12),
    height=600,
    width=800,
    margin=dict(t=50, l=50, r=50, b=50)
)

# Exibir o gráfico
fig.show()

Dia com menos defeitos

In [148]:
df_reset.loc[df_reset[['Total de Defeitos no Dia']].idxmin(), :]

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Gotas cortadas (toneladas),Total de Defeitos no Dia,Defeito Mais Frequente do Dia,...,Bolha aberta interna,Cascão cortante,Espessura abaixo do mínimo,Fundo deformado,Bolha na alça,Ferrugem no corpo,Fagulha,Alça deformada,Instabilidade,Crú
190,194057,C1164.0000R,2024-06-21,2151.84,168.5,29.3,15.9,22.194,79,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0


Ordem 194057

In [154]:
ordem_194057 = df_reset.loc[df_reset['Ordem de prod'] == 194057, :]
ordem_194057

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Gotas cortadas (toneladas),Total de Defeitos no Dia,Defeito Mais Frequente do Dia,...,Bolha aberta interna,Cascão cortante,Espessura abaixo do mínimo,Fundo deformado,Bolha na alça,Ferrugem no corpo,Fagulha,Alça deformada,Instabilidade,Crú
188,194057,C1164.0000R,2024-06-19,2148.42,169.6,80.0,39.5,22.024,193,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0
189,194057,C1164.0000R,2024-06-20,2175.05,169.5,79.3,42.2,22.022,196,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0
190,194057,C1164.0000R,2024-06-21,2151.84,168.5,29.3,15.9,22.194,79,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,0


> Gotas cortadas é do forno no dia, alimentando várias linhas. Dificulta a compreensão da relação das gotas cortadas por defeitos no dia de **uma linha**.

### Relação Gotas Cortadas e Total de Defeitos

In [144]:
px.scatter(data_frame=df_reset, 
           x='Gotas cortadas (toneladas)', 
           y='Total de Defeitos no Dia', 
           color='Total de Defeitos no Dia', 
           color_continuous_scale='peach', 
           size='Total de Defeitos no Dia')

# Solução - Prever o total de defeitos para nova ordem e total defeitos por dia

### Agrupar os dados por 'Ordem de prod' e 'Data de prod', somando os defeitos diários


In [79]:
df_sazonalidade = df_reset.groupby(['Ordem de prod', 'Data de prod'])[['Total de Defeitos no Dia']].sum().reset_index()

### Converter a coluna 'Data de prod' para o tipo datetime

In [80]:
df_sazonalidade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Ordem de prod             200 non-null    int64 
 1   Data de prod              200 non-null    object
 2   Total de Defeitos no Dia  200 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ KB


In [81]:
df_sazonalidade['Data de prod'] = pd.to_datetime(df_sazonalidade['Data de prod'])

### Criar novas colunas com informações sobre a data

In [82]:
df_sazonalidade['Dia da Semana'] = df_sazonalidade['Data de prod'].dt.day_name()  # Nome do dia da semana
df_sazonalidade['Mês'] = df_sazonalidade['Data de prod'].dt.month  # Número do mês
df_sazonalidade['Dia do Ano'] = df_sazonalidade['Data de prod'].dt.dayofyear  # Número do dia do ano

In [83]:
df_sazonalidade

,Ordem de prod,Data de prod,Total de Defeitos no Dia,Dia da Semana,Mês,Dia do Ano
0,192970,2024-01-03,222,Wednesday,1,3
1,192970,2024-01-04,249,Thursday,1,4
2,192970,2024-01-05,268,Friday,1,5
3,192970,2024-01-06,242,Saturday,1,6
4,192970,2024-01-07,232,Sunday,1,7
...,...,...,...,...,...,...
195,194121,2024-06-26,258,Wednesday,6,178
196,194121,2024-06-27,218,Thursday,6,179
197,194121,2024-06-28,218,Friday,6,180
198,194121,2024-06-29,213,Saturday,6,181


### Calcular a duração de produção (número de dias únicos) por 'Ordem de prod'

In [84]:
duracao_por_ordem = df_sazonalidade.groupby('Ordem de prod')['Data de prod'].nunique().reset_index()

In [85]:
duracao_por_ordem

,Ordem de prod,Data de prod
0,192970,9
1,193112,3
2,193164,12
3,193206,4
4,193207,10
5,193208,2
6,193209,7
7,193342,20
8,193368,7
9,193459,3


### Renomear a coluna resultante para 'Dias de Duração'

In [86]:
duracao_por_ordem.columns = ['Ordem de prod', 'Dias de Duração']

### Agrupar os dados por 'Ordem de prod' e somar os defeitos diários

In [87]:
defeitos_por_ordem = df_sazonalidade.groupby('Ordem de prod')['Total de Defeitos no Dia'].sum().reset_index()

### Renomear a coluna resultante para 'Total de Defeitos por Ordem'

In [88]:
defeitos_por_ordem.columns = ['Ordem de prod', 'Total de Defeitos por Ordem']

### Juntar as informações de 'Duracao por Ordem' e 'Defeitos por Ordem' em um único DataFrame

In [89]:
duracao_por_ordem = duracao_por_ordem.merge(defeitos_por_ordem, on='Ordem de prod', how='left')

In [90]:
duracao_por_ordem

,Ordem de prod,Dias de Duração,Total de Defeitos por Ordem
0,192970,9,2148
1,193112,3,584
2,193164,12,2661
3,193206,4,1041
4,193207,10,2130
5,193208,2,369
6,193209,7,1807
7,193342,20,3685
8,193368,7,1464
9,193459,3,505


### Gerar um gráfico de dispersão (scatter) com as informações de duração e defeitos

In [91]:
px.scatter(data_frame=duracao_por_ordem, 
           x='Dias de Duração', 
           y='Total de Defeitos por Ordem', 
           color='Total de Defeitos por Ordem', 
           color_continuous_scale='peach', 
           size='Total de Defeitos por Ordem')

### Adicionar as informações de 'Duração por Ordem' no DataFrame original 'df_sazonalidade'

In [92]:
df_sazonalidade = df_sazonalidade.merge(duracao_por_ordem, on='Ordem de prod', how='left')

In [93]:
# Exibir o DataFrame final
df_sazonalidade

,Ordem de prod,Data de prod,Total de Defeitos no Dia,Dia da Semana,Mês,Dia do Ano,Dias de Duração,Total de Defeitos por Ordem
0,192970,2024-01-03,222,Wednesday,1,3,9,2148
1,192970,2024-01-04,249,Thursday,1,4,9,2148
2,192970,2024-01-05,268,Friday,1,5,9,2148
3,192970,2024-01-06,242,Saturday,1,6,9,2148
4,192970,2024-01-07,232,Sunday,1,7,9,2148
...,...,...,...,...,...,...,...,...
195,194121,2024-06-26,258,Wednesday,6,178,6,1436
196,194121,2024-06-27,218,Thursday,6,179,6,1436
197,194121,2024-06-28,218,Friday,6,180,6,1436
198,194121,2024-06-29,213,Saturday,6,181,6,1436


### Criando Dataframe agrupado por ordem de produção, com operações diferentes

In [94]:
defeitos_por_duracao = df_sazonalidade.groupby('Ordem de prod').agg({'Total de Defeitos no Dia': 'sum', 'Dias de Duração': 'first'})

### Renomear a coluna 'Total de Defeitos no Dia' para 'Total de Defeitos por Ordem'

In [95]:
defeitos_por_duracao.rename(columns={'Total de Defeitos no Dia': 'Total de Defeitos por Ordem'}, inplace=True)

In [96]:
defeitos_por_duracao

,Total de Defeitos por Ordem,Dias de Duração
Ordem de prod,,
192970,2148,9
193112,584,3
193164,2661,12
193206,1041,4
193207,2130,10
193208,369,2
193209,1807,7
193342,3685,20
193368,1464,7


## Média de defeitos por dias da semana

### Agrupando dados por 'Dia da Semana' e calculando médias

In [97]:
df_medias_dia_semana = df_sazonalidade.groupby('Dia da Semana')[['Total de Defeitos no Dia']].mean().sort_values('Total de Defeitos no Dia', ascending=False).round(2)

In [98]:
fig = px.bar(df_medias_dia_semana.reset_index(), 
             x='Dia da Semana', y='Total de Defeitos no Dia',
             text='Total de Defeitos no Dia')

fig.update_traces(textposition='outside', texttemplate='%{text}', 
                  textfont_size=14, textfont_color='black')

fig.show()

## Média de defeitos por dia de produção

### Calcular a duração total de cada ordem

In [99]:
duracao_por_ordem = df_sazonalidade.groupby('Ordem de prod')['Data de prod'].nunique().reset_index()
duracao_por_ordem.columns = ['Ordem de prod', 'Dias de Duração']

### Merge com o DataFrame original

In [100]:
df_sazonalidade = df_sazonalidade.merge(duracao_por_ordem, on='Ordem de prod', how='left')

### Remover a coluna duplicada (se já havia 'Dias de Duração' no DataFrame original)

In [101]:
df_sazonalidade = df_sazonalidade.drop(columns=['Dias de Duração_x'])

### Renomear a coluna restante, se necessário

In [102]:
df_sazonalidade = df_sazonalidade.rename(columns={'Dias de Duração_y': 'Dias de Duração'})

### Padronizando a duração das ordens

Calcular o Dia Contado por Ordem

In [103]:
df_sazonalidade['Dia Contado'] = df_sazonalidade.groupby('Ordem de prod').cumcount() + 1

`.cumcount()`: Gera uma contagem cumulativa para cada linha dentro de cada grupo. A contagem começa em 0 para a primeira linha de cada grupo, 1 para a segunda linha, e assim por diante.


`+ 1`: Incrementa a contagem para começar em 1 em vez de 0, já que provavelmente você quer um número sequencial que faça mais sentido (como o "primeiro dia" de produção em vez de "dia 0").

In [104]:
df_sazonalidade[['Data de prod', 'Dia Contado']]

,Data de prod,Dia Contado
0,2024-01-03,1
1,2024-01-04,2
2,2024-01-05,3
3,2024-01-06,4
4,2024-01-07,5
...,...,...
195,2024-06-26,2
196,2024-06-27,3
197,2024-06-28,4
198,2024-06-29,5


Calcular o Dia Normalizado

In [105]:
df_sazonalidade['Dia Normalizado'] = df_sazonalidade['Dia Contado'] / df_sazonalidade['Dias de Duração']

`Dia Contado`: Representa o dia específico dentro de uma ordem de produção (e.g., dia 1, dia 2...).

`Dias de Duração`: É o número total de dias necessários para completar a ordem de produção.

`Dia Contado `/ `Dias de Duração`: Faz com que cada dia seja representado como uma fração do ciclo total de produção.

Visualização

In [106]:
df_sazonalidade[['Ordem de prod', 'Dia Contado', 'Dias de Duração', 'Dia Normalizado']]

,Ordem de prod,Dia Contado,Dias de Duração,Dia Normalizado
0,192970,1,9,0.111111
1,192970,2,9,0.222222
2,192970,3,9,0.333333
3,192970,4,9,0.444444
4,192970,5,9,0.555556
...,...,...,...,...
195,194121,2,6,0.333333
196,194121,3,6,0.500000
197,194121,4,6,0.666667
198,194121,5,6,0.833333


- Isso é útil para comparar dados de ordens de diferentes durações. Sem essa normalização, ordens mais curtas ou mais longas não seriam diretamente comparáveis.

- O resultado da coluna 'Dia Normalizado' estará sempre entre 0 e 1, onde:

`0`: Representa o início da produção (primeiro dia).

`1`: Representa o final da produção (último dia).

Valores intermediários (e.g., 0.5) indicam a posição relativa no ciclo.

### Calcular a média de defeitos por Dia Normalizado

In [107]:
defeitos_por_dia_normalizado = df_sazonalidade.groupby('Dia Normalizado')[['Total de Defeitos no Dia']].mean().reset_index()
defeitos_por_dia_normalizado.columns = ['Dia Normalizado', 'Média de Defeitos']

Visualização

In [108]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=defeitos_por_dia_normalizado['Dia Normalizado'],
                         y=defeitos_por_dia_normalizado['Média de Defeitos'],
                         mode='lines+markers', 
                         marker=dict(size=10),
                         line=dict(color='red'),
                         name='Média de Defeitos'))

fig.update_layout(
    title='Média de Defeitos por Dia Normalizado',
    title_font_size=16,
    xaxis_title='Dia Normalizado',
    xaxis_title_font_size=14,
    yaxis_title='Média de Defeitos',
    yaxis_title_font_size=14
)

fig.show()

### Calcular a média de defeitos por Dia Contado

Agrupar pelos valores inteiros de Dia Contado

In [109]:
defeitos_por_dia_contado = df_sazonalidade.groupby('Dia Contado')[['Total de Defeitos no Dia']].mean().reset_index()
defeitos_por_dia_contado.columns = ['Dia Contado', 'Média de Defeitos']

Visualização

In [110]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=defeitos_por_dia_contado['Dia Contado'],
                         y=defeitos_por_dia_contado['Média de Defeitos'],
                         mode='lines+markers', 
                         marker=dict(size=10),
                         line=dict(color='red'),
                         name='Média de Defeitos'))

fig.update_layout(
    title='Média de Defeitos por Dia Contado',
    title_font_size=16,
    xaxis_title='Dia Contado',
    xaxis_title_font_size=14,
    yaxis_title='Média de Defeitos',
    yaxis_title_font_size=14
)

fig.show()

## Implementação do Modelo de Regressão

### Resetando index

In [111]:
df_total_duracao_reset = defeitos_por_duracao.reset_index()

In [112]:
df_total_duracao_reset.head()

,Ordem de prod,Total de Defeitos por Ordem,Dias de Duração
0,192970,2148,9
1,193112,584,3
2,193164,2661,12
3,193206,1041,4
4,193207,2130,10


### Passo 1: Preparar os dados para o total de defeitos e a duração da ordem

In [113]:
df_combined = df_reset.merge(df_total_duracao_reset[['Ordem de prod', 'Total de Defeitos por Ordem', 'Dias de Duração']], on='Ordem de prod', how='left')

### Passo 2: Separar variáveis independentes (medidas e duração) e dependente (total de defeitos)

In [114]:
X = df_combined[['Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro', 'Dias de Duração']]
y = df_combined['Total de Defeitos por Ordem']

### Passo 3: Dividir os dados em treino e teste

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Passo 4: Normalizar os dados

In [116]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Passo 5: Treinar o modelo para prever o total de defeitos

In [117]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train_scaled, y_train)

GradientBoostingRegressor(random_state=42)

### Passo 6: Previsões de total de defeitos

In [118]:
y_pred_gb = gb_model.predict(X_test_scaled)

### Passo 7: Avaliar a performance do modelo

In [119]:
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print(f"Gradient Boosting - Mean Squared Error (MSE): {mse_gb}")
print(f"Gradient Boosting - R² Score: {r2_gb}")

Gradient Boosting - Mean Squared Error (MSE): 7698.757546270441
Gradient Boosting - R² Score: 0.9918282458806885


- **Mean Squared Error (MSE)**: Mede o erro médio ao quadrado entre os valores previstos pelo modelo e os valores reais. No caso, um MSE de 7698,76 indica que, em média, o modelo apresenta esse nível de erro nas previsões, o que pode ser considerado aceitável dependendo da escala dos defeitos totais.


- **R² Score**: Representa o quanto do comportamento dos dados o modelo consegue explicar, com valores próximos de 1,0 indicando alta precisão. Aqui, o R² de 0,99 mostra que o modelo explica 99% da variação nos defeitos totais, o que reflete um excelente ajuste aos dados.

> Meu modelo de Gradient Boosting está com um R² de 0,99, o que indica alta precisão nas previsões, embora haja sinais de ***overfitting*** devido ao impacto significativo da ***duração*** da ordem nos defeitos ***totais***. Apesar disso, o diferencial da solução está na capacidade de ***distribuir*** os defeitos totais previstos ao longo dos dias da nova ordem, permitindo identificar os dias críticos e agir preventivamente para otimizar a produção. 

## Testando modelo

### Criando nova ordem e prevendo com o modelo

In [120]:
nova_ordem = {
    'Peso médio (g)': 160,
    'Comprimento médio (mm)': 120,
    'Diâmetro': 30.2,
    'Dias de Duração': 15
}

X_nova_ordem = scaler.transform(pd.DataFrame([nova_ordem])) 
total_defeitos_previsto = gb_model.predict(X_nova_ordem)
print(f"Total de defeitos previsto para a nova ordem: {total_defeitos_previsto[0]}")

Total de defeitos previsto para a nova ordem: 2331.3429814026454


### Calcular a soma total dos defeitos para garantir que a soma seja igual ao total esperado

In [121]:
total_medio_defeitos = defeitos_por_dia_contado['Média de Defeitos'].sum()

- O que faz: Calcula a soma total da coluna 'Média de Defeitos', ou seja, o valor agregado da média de defeitos registrados por dia.

- Por que isso é útil: Este valor será usado para determinar a contribuição proporcional de cada dia na previsão do total de defeitos.

### Garantir que a soma dos defeitos seja proporcionalmente ajustada ao total de defeitos previsto

In [122]:
defeitos_por_dia_contado['Defeitos Previstos'] = (defeitos_por_dia_contado['Média de Defeitos'] / total_medio_defeitos) * total_defeitos_previsto

- O que faz: Estima a quantidade de defeitos previstos para cada dia com base na média de defeitos já registrada e o valor total de defeitos previstos.

**Passo a passo do cálculo**:

1.  `defeitos_por_dia_contado['Média de Defeitos']` / `total_medio_defeitos`: Calcula a proporção de defeitos de cada dia em relação ao total médio de defeitos.

2. Multiplica essa proporção pelo `total_defeitos_previsto` para distribuir os defeitos previstos entre os dias, proporcionalmente à média de defeitos observada.

- Por que isso é útil: Permite projetar o total de defeitos em cada dia com base em dados históricos.

### Recalcular a distribuição dos defeitos considerando o número de dias de duração

In [123]:
dias_duracao = nova_ordem['Dias de Duração']

### Selecionar as previsões apenas para os dias da nova ordem

In [124]:
df_dias_duracao = defeitos_por_dia_contado.head(dias_duracao).copy()  # Usando .copy() para evitar aviso

### Garantir que a soma dos defeitos nos dias da nova ordem seja igual ao total de defeitos previsto


Ajustar a distribuição proporcionalmente para que a soma dos defeitos seja igual ao `total_defeitos_previsto`

In [125]:
soma_defeitos_dia = df_dias_duracao['Defeitos Previstos'].sum()
ajuste_fator = total_defeitos_previsto / soma_defeitos_dia

# Ajustar os defeitos previstos diretamente no DataFrame com .loc[]
df_dias_duracao.loc[:, 'Defeitos Previstos'] = df_dias_duracao['Defeitos Previstos'] * ajuste_fator

- O que faz:

`soma_defeitos_dia`: Soma os valores da coluna 'Defeitos Previstos' no DataFrame `df_dias_duracao`.

`ajuste_fator`: Calcula o fator necessário para ajustar a soma total dos defeitos previstos ao valor exato de `total_defeitos_previsto`.

Multiplica cada valor da coluna 'Defeitos Previstos' pelo `ajuste_fator` para garantir que a soma total da coluna corresponda ao valor exato de `total_defeitos_previsto`.

- Por que isso é útil: Durante o cálculo inicial de `'Defeitos Previstos'`, pode haver pequenas discrepâncias de arredondamento que fazem com que a soma dos valores não corresponda exatamente ao `total_defeitos_previsto`. Esse ajuste corrige isso.

### Visualização Geral

In [126]:
print(f"Total de defeitos previsto para a nova ordem: {total_defeitos_previsto[0]:.2f}")

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_dias_duracao['Dia Contado'],
                         y=df_dias_duracao['Defeitos Previstos'],
                         mode='lines+markers',
                         marker=dict(size=10),
                         line=dict(color='green'),
                         name='Defeitos Previstos'))

fig.update_layout(
    title=f'Previsão de Defeitos ao Longo dos {dias_duracao} Dias da Nova Ordem',
    title_font_size=16,
    xaxis_title='Dia de Produção',
    xaxis_title_font_size=14,
    yaxis_title='Defeitos Previstos',
    yaxis_title_font_size=14
)

fig.show()

df_dias_duracao[['Dia Contado', 'Defeitos Previstos']].set_index('Dia Contado')

Total de defeitos previsto para a nova ordem: 2331.34


,Defeitos Previstos
Dia Contado,
1,172.432072
2,189.648274
3,166.055019
4,170.549168
5,166.337233
6,164.986433
7,144.865521
8,155.170861
9,165.541841


## Próximo Passo: Analisar 2º dia de produção das ordens

### Fazer o merge entre o df_reset e o duracao_por_ordem com base na 'Ordem de prod'

In [127]:
df_reset_merged = df_reset.merge(duracao_por_ordem, on='Ordem de prod', how='left')

### Garantir que 'Data de prod' seja do tipo datetime

In [128]:
df_reset_merged['Data de prod'] = pd.to_datetime(df_reset_merged['Data de prod'])

### Agrupar por 'Ordem de prod' e calcular o 'Dia Contado' para cada grupo

In [129]:
df_reset_merged['Dia Contado'] = df_reset_merged.groupby('Ordem de prod')['Data de prod'].transform(lambda x: (x - x.min()).dt.days + 1)

### Agrupando medidas por dias contados (padronizados) 

In [130]:
df_medidas_por_dias = df_reset_merged.groupby('Dia Contado')[['Temperatura (F°)', 'Peso médio (g)', 'Diâmetro', 'Comprimento médio (mm)']].mean()
df_medidas_por_dias_reset = df_medidas_por_dias.reset_index()

Visualização

In [131]:
px.line(data_frame=df_medidas_por_dias_reset, x='Dia Contado', y='Temperatura (F°)')

### Analisando dias 2 da ordem

Localizando dias 2

In [132]:
ordens_dia_2 = df_reset_merged.loc[df_reset_merged['Dia Contado' ] == 2, :]

In [133]:
ordens_dia_2

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Gotas cortadas (toneladas),Total de Defeitos no Dia,Defeito Mais Frequente do Dia,...,Espessura abaixo do mínimo,Fundo deformado,Bolha na alça,Ferrugem no corpo,Fagulha,Alça deformada,Instabilidade,Crú,Dias de Duração,Dia Contado
1,192970,C0516.0000R,2024-01-04,2118.92,169.7,85.6,18.1,21.022,249,Sujo de óleo,...,0,0,0,0,0,0,0,0,9,2
10,193112,C1164.0000R,2024-01-02,2189.08,168.3,77.6,41.6,21.857,203,Sujo de óleo,...,0,0,0,0,0,0,0,0,3,2
13,193164,C1280.0000R,2024-01-12,2128.92,148.5,95.2,35.7,20.045,240,Sujo de óleo,...,0,0,0,0,0,0,0,0,12,2
25,193206,C1284.0000R,2024-01-26,2177.37,171.2,91.9,38.6,21.226,255,Sujo de óleo,...,0,0,0,0,0,0,0,0,4,2
29,193207,C1298.0000R,2024-01-29,2144.11,170.6,99.8,37.1,14.741,243,Falta de seção,...,0,0,0,0,0,0,0,0,10,2
39,193208,C0104.0000R,2024-02-08,2171.88,122.2,46.7,20.7,5.292,242,Fora de cor,...,0,0,0,0,0,0,0,0,2,2
41,193209,C0509.0000R,2024-02-09,2128.06,191.0,87.3,18.6,21.495,305,Sujo de óleo,...,0,0,0,0,0,0,0,0,7,2
48,193342,C0516.0000R,2024-02-22,2126.32,171.2,107.6,28.1,21.936,296,Sujo de óleo,...,0,0,0,0,0,0,0,0,20,2
75,193459,C0068.0000R,2024-03-12,2147.79,194.2,95.9,40.1,16.788,189,Sujo de óleo,...,0,0,0,0,0,0,0,0,3,2
78,193462,C1284.0000R,2024-03-22,2173.72,172.6,86.0,40.4,22.363,265,Trinca por choque térmico,...,0,0,0,0,0,0,0,0,5,2
